<a href="https://colab.research.google.com/github/KimJae-hee/kyakyakyakya/blob/master/%ED%8C%8C%EB%9D%BC%ED%8C%8C%EB%9D%BC%ED%8C%8C%EB%9D%BC%ED%8C%8C%EB%9D%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import numpy as np
import pandas as pd
from sklearn.metrics import log_loss
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split

!pip install xgboost
!pip install category_encoders
import xgboost as xgb
import category_encoders
import time

In [8]:
tr = pd.read_csv("/content/drive/My Drive/P-SAT/2020-2/방학 세미나/기존 데이터/Train_data.csv", engine = "python")
te = pd.read_csv("/content/drive/My Drive/P-SAT/2020-2/방학 세미나/기존 데이터/Test_data.csv")

In [9]:
tr["event_datetime"] = pd.to_datetime(tr["event_datetime"])
tr["hour"] = tr["event_datetime"].dt.hour
te["event_datetime"] = pd.to_datetime(te["event_datetime"])
te["hour"] = te["event_datetime"].dt.hour
tr['hour'] = tr['hour'].astype(str)
te['hour'] = te['hour'].astype(str)
tr_y = tr['click']
te_bid_id = te["bid_id"]
tr_x = tr.drop(['bid_id', 'click',"event_datetime",'ssp_id','device_os','media_domain','campaign_id','placement_type','advertisement_id'],axis = 1)
te = te.drop(['bid_id', "event_datetime",'ssp_id','device_os','media_domain','campaign_id','placement_type','advertisement_id'],axis = 1)

#파라미터 튜닝 시 생략

In [19]:
encoder = category_encoders.CatBoostEncoder()
train_x, val_x, train_y, val_y = train_test_split(tr_x, tr_y, test_size = 0.2, train_size = 0.8, stratify = tr_y, random_state = 1234)
train_cbe = encoder.fit_transform(train_x,train_y)
val_cbe = encoder.transform(val_x)
eval_set = [(val_cbe, val_y)]
model = xgb.XGBClassifier(learning_rate = 0.1, max_depth = 5, min_child_weight = 5, gamma = 0.3, objective = 'binary:logistic', n_estimators = 500)
model.fit(train_cbe, train_y, eval_metric = 'logloss', eval_set = eval_set, early_stopping_rounds= 50)

[0]	validation_0-logloss:0.623201
Will train until validation_0-logloss hasn't improved in 50 rounds.
[1]	validation_0-logloss:0.567539
[2]	validation_0-logloss:0.519552
[3]	validation_0-logloss:0.481011
[4]	validation_0-logloss:0.447475
[5]	validation_0-logloss:0.418007
[6]	validation_0-logloss:0.393258
[7]	validation_0-logloss:0.371968
[8]	validation_0-logloss:0.354008
[9]	validation_0-logloss:0.338387
[10]	validation_0-logloss:0.324763
[11]	validation_0-logloss:0.312366
[12]	validation_0-logloss:0.301884
[13]	validation_0-logloss:0.292727
[14]	validation_0-logloss:0.284346
[15]	validation_0-logloss:0.277466
[16]	validation_0-logloss:0.271448
[17]	validation_0-logloss:0.265609
[18]	validation_0-logloss:0.260895
[19]	validation_0-logloss:0.256739
[20]	validation_0-logloss:0.25344
[21]	validation_0-logloss:0.250216
[22]	validation_0-logloss:0.247271
[23]	validation_0-logloss:0.244835
[24]	validation_0-logloss:0.242738
[25]	validation_0-logloss:0.240803
[26]	validation_0-logloss:0.23929

ValueError: ignored

In [22]:
te_cbe = encoder.transform(te)
pred = model.predict_proba(te_cbe)

In [23]:
submit_col_click = pd.DataFrame(pred)[1].values
submit_col_bid_id = te_bid_id
submit = pd.DataFrame({'bid_id' : te_bid_id, 'click': submit_col_click})

In [24]:
submit

,bid_id,click
0,Sh0wWllIpg,0.004170
1,gKh5ZZcIAu,0.050120
2,rwnUHAWIck,0.003558
3,0Eootr00Gl,0.050410
4,arSLZs3c4o,0.025077
...,...,...
482791,Y9gXqBuS2c,0.005613
482792,ndtgp2Pjvo,0.004268
482793,yj3QDK8Yyr,0.267389
482794,mPcwUjvR6R,0.010484


In [25]:
submit.to_csv('/content/drive/My Drive/P-SAT/2020-2/방학 세미나/시범용submit3.csv', index = False)

# 여기부터는 파라미터 튜닝!

In [10]:
X_use, _, Y_use, _ = train_test_split(tr_x, tr_y, test_size = 0.9, train_size = 0.1, stratify = tr_y, random_state = 1234)

In [11]:
kfold  = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 1234)

In [ ]:
lr_list = [0.07, 0.1]
max_list = [6, 8, 10]
encoder_list = [category_encoders.OrdinalEncoder(), category_encoders.CatBoostEncoder(), category_encoders.HelmertEncoder()] 
subsample_list = [0.4, 0.6, 0.8]
result_df = pd.DataFrame(columns=["learning_rate", "max_depth","encoder", "subsample", 'epoch', "time_cost", "result"])
for lr in lr_list:
  for max in max_list:
    for num, encoder in enumerate(encoder_list):
      for subsample in subsample_list:
        result_list = []
        time_list = []
        iteration_list = []
        for train_idx, test_idx in kfold.split(X_use, Y_use):
          train_fold_X, test_fold_X = X_use.iloc[train_idx,:], X_use.iloc[test_idx,:]
          train_fold_Y, test_fold_Y = Y_use.iloc[train_idx], Y_use.iloc[test_idx]
          train_x, val_x, train_y, val_y = train_test_split(train_fold_X, train_fold_Y, test_size = 0.2, train_size = 0.8, stratify = train_fold_Y, random_state = 1234)

          train_cbe = encoder.fit_transform(train_x, train_y)
          val_cbe = encoder.transform(val_x)

          eval_set = [(val_cbe, val_y)]

          model = xgb.XGBClassifier(random_state = 1234, n_estimators = 600, learning_rate = lr, max_depth = max, subsample = subsample, min_child_weight = 5, gamma = 0.3)
          model.fit(train_cbe, train_y, eval_metric = "logloss", eval_set = eval_set, early_stopping_rounds = 30, verbose = 100)
          
          start = time.time()
          test_cbe = encoder.transform(test_fold_X)
          pred_fold = model.predict_proba(test_cbe)
          end = time.time()
          time_cost = end - start
          time_list.append(time_cost)

          result_tmp = log_loss(test_fold_Y, pred_fold)
          result_list.append(result_tmp)
          
          iter = model.best_iteration
          iteration_list.append(iter)

        print(f"learning_rate : {lr}, max_depth: {max}, encoder: {num}, iteration : {np.mean(iteration_list)}, :::::::::: {np.mean(result_list)}")
        df_tmp = pd.DataFrame([[lr, max, num, subsample, np.mean(iteration_list), np.mean(time_list), np.mean(result_list)]], columns=["learning_rate", "max_depth", "encoder", "subsample", 'epoch', "time_cost", "result"])
        result_df = result_df.append(df_tmp)
        result_df.to_csv("/content/drive/My Drive/P-SAT/2020-2/방학 세미나/파라미터 튜닝/20200820.csv", index = False)

[0]	validation_0-logloss:0.644879
Will train until validation_0-logloss hasn't improved in 30 rounds.
[100]	validation_0-logloss:0.242132
Stopping. Best iteration:
[161]	validation_0-logloss:0.241741

[0]	validation_0-logloss:0.644692
Will train until validation_0-logloss hasn't improved in 30 rounds.
[100]	validation_0-logloss:0.24045
[200]	validation_0-logloss:0.239744
Stopping. Best iteration:
[188]	validation_0-logloss:0.239704

[0]	validation_0-logloss:0.644797
Will train until validation_0-logloss hasn't improved in 30 rounds.
[100]	validation_0-logloss:0.240451
[200]	validation_0-logloss:0.239432
Stopping. Best iteration:
[256]	validation_0-logloss:0.2392

[0]	validation_0-logloss:0.644744
Will train until validation_0-logloss hasn't improved in 30 rounds.
[100]	validation_0-logloss:0.240373
[200]	validation_0-logloss:0.239837
Stopping. Best iteration:
[223]	validation_0-logloss:0.239738

[0]	validation_0-logloss:0.644728
Will train until validation_0-logloss hasn't improved in 

In [25]:
 result_df.to_csv("/content/drive/My Drive/P-SAT/2020-2/방학 세미나/파라미터 튜닝/20200820.csv", index = False)

In [23]:
result_df

,learning_rate,max_depth,encoder,subsample,epoch,time_cost,result
0,0.07,6,0,0.4,206.4,-1.014124,0.239119
0,0.07,6,0,0.6,192.4,-0.945029,0.238680
